In [1]:
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
from sklearn import datasets
import plotly as plt
import plotly.express as px
from features import *
from clustering import *
from utils import *
from constant import  PATH_OUTPUT, MODEL_CLUSTERING, PATH_DATA, PATH_DATA_ALL

from sklearn.cluster import KMeans

from sklearn.pipeline import Pipeline

from skimage.transform import resize

In [2]:
from images import load_images_from_folder

# Example usage:
folder_path = PATH_DATA_ALL + "/code_test"
images, labels_true, folder_names, smallest_height, smallest_width, smallest_height2, smallest_width2 = load_images_from_folder(folder_path)
taille = len(images)
nombre_de_canaux = 3
# print(f"Smallest height: {smallest_height}")
# print(f"Smallest width: {smallest_width}")
# print(f"Hightest height: {smallest_height2}")
# print(f"Hightest width: {smallest_width2}")

In [3]:

# Supposons que `images` est votre liste d'images en RGB
descriptors_hsv = convert_color_space(images, "HSV") # ou "Lab"

In [4]:
# descriptors_sift = compute_sift_descriptors(images)
sift_descriptors = extract_sift_features(descriptors_hsv)
print(f"Taille de sift_descriptors: {len(sift_descriptors)}")
# Étape 2 : Création des vecteurs de caractéristiques avec Bag of Features
descriptors_sift = create_bag_of_features(sift_descriptors, n_clusters=20)

Taille de sift_descriptors: 42


In [5]:
images_to_use = descriptors_hsv

In [6]:

descriptors_hog = compute_hog_descriptors(images_to_use)

In [7]:
descriptors_hist = compute_gray_histograms(images_to_use)

**5. Conversion des données de clustering au format requis pour la visualisation avec Streamlit.**

**TODO :**
- Dans le fichier `utils.py`, implémenter la fonction `conversion_3d` afin de convertir un vecteur de dimension n vers une dimension 3 pour a visualisation.
- Lien : https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

In [8]:
print(np.array(descriptors_hog).shape)
print(np.array(descriptors_hsv).shape)
print(np.array(descriptors_sift).shape)

(42, 8192)
(42, 256, 256, 3)
(42, 20)


**3. Application de RBM et KMEANS**

In [9]:
from sklearn.preprocessing import StandardScaler

descriptors = descriptors_hist

# Présumons que `images` est votre liste d'images prétraitées et aplatie en vecteurs

# Initialisation de la classe StackedRBM
stacked_rbm = StackedRBM(n_components_list=[256, 128], n_iter=10, learning_rate=0.01, batch_size=10)

# Ajustement des RBMs sur les données d'image
stacked_rbm.fit(descriptors)

# Transformation des images en nouvelles représentations avec les RBMs entraînés
transformed_images = stacked_rbm.transform(descriptors)

# Normalisation des caractéristiques pour améliorer les performances de K-Means
scaler = StandardScaler()
transformed_images_scaled = scaler.fit_transform(transformed_images)

# Clustering avec K-Means
kmeans = KMeans(n_clusters=20, random_state=42)
clusters = kmeans.fit_predict(transformed_images_scaled)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -13.74, time = 0.03s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -6.68, time = 0.02s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -4.24, time = 0.05s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -2.81, time = 0.03s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -3.05, time = 0.04s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -2.18, time = 0.02s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -1.97, time = 0.02s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -1.57, time = 0.02s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -1.23, time = 0.02s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -1.37, time = 0.02s
[BernoulliRBM] Iteration 1, pseudo-likelihood = -186.17, time = 0.02s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -178.84, time = 0.03s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -175.87, time = 0.01s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -174.95, time = 0.01s
[BernoulliRBM] Iteration 5, pseudo-lik

c:\Users\comma\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:1474: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (20). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [10]:
print("\n\n##### Résultat ######")
# metric_hist = show_metric(labels_true, clusters, transformed_images_scaled, bool_show=True, name_descriptor="HISTOGRAM", bool_return=True)

metric_hsv_hist_rbm = show_metric(labels_true, clusters, transformed_images_scaled, bool_show=True, name_descriptor="HSV et HISTOGRAM", name_model = "Stacked RBM", bool_return=True)
# metric_hsv_hog_rbm = pd.read_excel("output/save_clustering_hsv_hog_rbm_kmeans.xlsx")
# metric_hsv_sift_rbm = pd.read_excel("output/save_clustering_hsv_sift_rbm_kmeans.xlsx")
# metric_hist_rbm = pd.read_excel("output/save_clustering_hist_rbm_kmeans.xlsx")
# metric_hog_rbm = pd.read_excel("output/save_clustering_hog_rbm_kmeans.xlsx")
# metric_sift_rbm = pd.read_excel("output/save_clustering_sift_rbm_kmeans.xlsx")



##### Résultat ######
########## Métrique descripteur : HSV_HISTOGRAM
Adjusted Mutual Information: -0.00973927291987954
Silhouette Score: 1.0


**5. Conversion des données de clustering au format requis pour la visualisation avec Streamlit.**

**TODO :**
- Dans le fichier `utils.py`, implémenter la fonction `conversion_3d` afin de convertir un vecteur de dimension n vers une dimension 3 pour a visualisation.
- Lien : https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

In [11]:
scaler = StandardScaler()
descriptors_hist_norm = scaler.fit_transform(descriptors_hist)
descriptors_hog_norm = scaler.fit_transform(descriptors_hog)

#conversion vers un format 3D pour la visualisation
x_3d_hist = conversion_3d(descriptors_hist_norm)
# x_3d_hog = conversion_3d(descriptors_hog_norm)

# création des dataframe pour la sauvegarde des données pour la visualisation
df_hist = create_df_to_export(x_3d_hist, labels_true, kmeans.labels_)
# df_hog = create_df_to_export(x_3d_hog, labels_true, kmeans.labels_)

# Vérifie si le dossier existe déjà
if not os.path.exists(PATH_OUTPUT):
    # Crée le dossier
    os.makedirs(PATH_OUTPUT)

# sauvegarde des données
df_hist.to_excel(PATH_OUTPUT+"/save_clustering_hsv_hist_rbm_kmeans.xlsx")
# df_hog.to_excel(PATH_OUTPUT+"/save_clustering_hog_kmeans.xlsx")




In [14]:
list_dict = [metric_hsv_hist_rbm]
df_metric = pd.DataFrame(list_dict)
df_metric.to_excel(PATH_OUTPUT+"/save_metric.xlsx")

SyntaxError: invalid syntax (1554332066.py, line 2)